# Image Viewer

The current version of the image viewer allows for efficient lookup of any of the thousands of DICOM images in the DSB 2017 dataset or RAW files in the associated LUNA16 dataset

**Ideas for Improvements:**
* For DSB Images, incorporate whether or not patient has cancer into the display (perhaps with red/green border?)
* For LUNA images, highlight labeled nodes (perhaps with circle?)

**Important Notes:**
* This script assumes that all LUNA files are one level below the LUNA16 folder in you file structure (and not in subfolders).  You may need to rearrange your file directories to conform to this expectation
* You will need to have the PyDICOM and SimpleITK libraries installed to run this script.  Run the following cell to install these libraries.

# DICOM Image Viewer

In [ ]:
%%cmd
pip install pydicom
pip install SimpleITK

In [ ]:
import os
import dicom
import SimpleITK as sitk
import numpy as np
from ipywidgets import widgets, interact, interactive, IntSlider
from IPython.display import display,clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Load list of all patients
dicom_path = 'E:\\DSB 2017\\stage1\\stage1' #Hard code the location of the data
dicom_patients = sorted(os.listdir(dicom_path))

In [ ]:
#Instantiate Controls
dicom_patient_list = widgets.Select(description = 'Patients',options = dicom_patients)  # populate the drop down with the sorted list of patients
dicom_img_slider = IntSlider(description = "Image Number", min = 0) # create image slider and set min = 0; we won't know the max until a patient is selected

In [ ]:
# Function to load a patient's data
def load_dicom_patient(args = ''):
    dicom_patient_path = os.path.join(dicom_path,dicom_patient_list.value)  #store the path to the selected patient
    global dicom_slices
    dicom_slices = [dicom.read_file(dicom_patient_path + '/' + s) for s in os.listdir(dicom_patient_path)] #read in the individual slices
    dicom_slices.sort(key = lambda x: int(x.ImagePositionPatient[2])) #sort slices based on vertical position
    dicom_img_slider.max = len(dicom_slices) - 1 #set the max of the image slider
    plot_dicom_img()

In [ ]:
def plot_dicom_img(args = ''):
    dicom_img = dicom_slices[dicom_img_slider.value].pixel_array #extract the image data from the selected slice
    clear_output(wait = True)
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.imshow(dicom_img,cmap=plt.cm.gray)
    ax.axes.axis('off')

In [ ]:
dicom_patient_list.observe(load_dicom_patient,'value')
dicom_img_slider.observe(plot_dicom_img,'value')

display(dicom_patient_list)
display(dicom_img_slider)

load_dicom_patient()

# RAW Image Viewer

In [ ]:
raw_path = 'E:\\LUNA16\\' #Hard code the location of the data
all_files = os.listdir(raw_path)
raw_patients = [i for i in all_files if '.mhd' in i]
raw_patients = sorted(raw_patients)

In [ ]:
raw_patient_list = widgets.Select(description = 'Patients',options = raw_patients)  # populate the drop down with the sorted list of patients
raw_img_slider = IntSlider(description = "Image Number", min = 0) # create image slider and set min = 0; we won't know the max until a patient is selected

In [ ]:
# Function to load a patient's data
def load_raw_patient(args = ''):
    raw_patient_path = os.path.join(raw_path,raw_patient_list.value)  #store the path to the selected patient
    global raw_slices
    raw_slices = sitk.ReadImage(raw_patient_path) 
    raw_slices = sitk.GetArrayFromImage(raw_slices)
    raw_img_slider.max = len(raw_slices) - 1 #set the max of the image slider
    plot_raw_img()

In [ ]:
def plot_raw_img(args = ''):
    raw_img = raw_slices[raw_img_slider.value] #extract the image data from the selected slice
    clear_output(wait = True)
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.imshow(raw_img,cmap=plt.cm.gray)
    ax.axes.axis('off')

In [ ]:
raw_patient_list.observe(load_raw_patient,'value')
raw_img_slider.observe(plot_raw_img,'value')

display(raw_patient_list)
display(raw_img_slider)

load_raw_patient()